In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess
from PIL import Image

# Correct Dataset Paths
dataset_paths = {
    "inverted": r"D:\project work 2 phase 2\fingerprint\augmented_data\inverted",
    "multi_augmented": r"D:\project work 2 phase 2\fingerprint\augmented_data\multi_augmented"
}

# Correct Label Paths
label_files = {
    "inverted": r"D:\project work 2 phase 2\fingerprint\augmented_data\inverted\inverted_labels.txt",
    "multi_augmented": r"D:\project work 2 phase 2\fingerprint\augmented_data\multi_augmented\multi_augmented_labels.txt"
}

# AlexNet Model Definition
def build_alexnet():
    input_layer = tf.keras.layers.Input(shape=(256, 256, 3))  
    x = Conv2D(96, (11, 11), strides=4, activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=2)(x)
    x = Conv2D(256, (5, 5), activation='relu', padding='same')(x)
    x = MaxPooling2D((3, 3), strides=2)(x)
    x = Conv2D(384, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(384, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((3, 3), strides=2)(x)
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1000, activation='softmax')(x)  

    model = Model(inputs=input_layer, outputs=x)  
    return model

# Load Pretrained Models
alexnet_model = build_alexnet()
alexnet_model.summary()  # Check model structure

googlenet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Function to check if a file is a valid image
def is_valid_image(img_path):
    try:
        with Image.open(img_path) as img:
            img.verify()  # Verify if it's a valid image
        return True
    except:
        return False

# Function to Load Labels from Text File
def load_labels(label_file):
    label_dict = {}  # Dictionary to store {image_name: (finger_index, user_id, augmentation_type, unique_id)}
    
    with open(label_file, "r") as file:
        for line in file:
            parts = line.strip().split(",")  # Split by comma (CSV format)
            if len(parts) >= 5:  # Ensure all values exist
                image_name = parts[0].strip()
                finger_index = int(parts[1].strip())
                user_id = int(parts[2].strip())
                augmentation_type = parts[3].strip()
                unique_id = int(parts[4].strip())
                
                label_dict[image_name] = (finger_index, user_id, augmentation_type, unique_id)
    
    return label_dict

# Function to Extract Features
def extract_features(model, dataset_path, img_size, preprocess_func, label_dict):
    feature_list = []
    image_labels = []  # Store labels matching extracted features
    
    # Get only valid image files that exist in the label dictionary (Maintaining Order)
    image_files = sorted([
        f for f in os.listdir(dataset_path) 
        if f in label_dict and f.lower().endswith(('.bmp', '.jpg', '.jpeg', '.png')) and is_valid_image(os.path.join(dataset_path, f))
    ])

    for img_name in image_files:
        img_path = os.path.join(dataset_path, img_name)
        img = image.load_img(img_path, target_size=(img_size, img_size))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_func(img_array)

        features = model.predict(img_array)
        features = features.flatten()
        feature_list.append(features)
        image_labels.append(label_dict[img_name])  # Store all label columns

    return np.array(feature_list), np.array(image_labels)

# Extract and Save Features with Labels
for dataset_type, dataset_path in dataset_paths.items():
    label_file = label_files[dataset_type]  # Get corresponding label file
    label_dict = load_labels(label_file)  # Load labels

    # Extract AlexNet Features
    alexnet_features, alexnet_labels = extract_features(alexnet_model, dataset_path, 256, lambda x: x / 255.0, label_dict)
    np.save(f"{dataset_type}_alexnet_features.npy", alexnet_features)
    np.save(f"{dataset_type}_alexnet_labels.npy", alexnet_labels)  # Save labels
    
    # Extract GoogleNet Features
    googlenet_features, googlenet_labels = extract_features(googlenet_model, dataset_path, 150, inception_preprocess, label_dict)
    np.save(f"{dataset_type}_googlenet_features.npy", googlenet_features)
    np.save(f"{dataset_type}_googlenet_labels.npy", googlenet_labels)  # Save labels

print("Feature extraction complete. Features and labels saved as .npy files.")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 62, 62, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 30, 30, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 30, 30, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 14, 14, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,281,344 (222.33 MB)

 Trainable params: 58,281,344 (222.33 MB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━

In [ ]:
# Function to load and display npy files
def display_npy(file_path, num_samples=5):
    data = np.load(file_path)  # Load the .npy file
    print(f"Shape of {file_path}: {data.shape}")  # Print shape of the data
    print(f"First {num_samples} samples:\n", data[:num_samples])  # Print first few samples

# File paths (Modify as needed)
dataset_types = ["inverted", "multi_augmented"]
feature_models = ["alexnet", "googlenet"]

for dataset in dataset_types:
    for model in feature_models:
        feature_file = f"{dataset}_{model}_features.npy"
        label_file = f"{dataset}_{model}_labels.npy"
        
        print(f"\n### Displaying Features for {dataset} - {model} ###")
        display_npy(feature_file)

        print(f"\n### Displaying Labels for {dataset} - {model} ###")
        display_npy(label_file)


### Displaying Features for inverted - alexnet ###
Shape of inverted_alexnet_features.npy: (1600, 4096)
First 5 samples:
 [[0.03183629 0.         0.         ... 0.         0.         0.0126222 ]
 [0.03285477 0.         0.         ... 0.         0.         0.01690636]
 [0.03109702 0.         0.         ... 0.         0.         0.01119474]
 [0.03229445 0.         0.         ... 0.         0.         0.00889634]
 [0.03041353 0.         0.         ... 0.         0.         0.01446172]]

### Displaying Labels for inverted - alexnet ###
Shape of inverted_alexnet_labels.npy: (1600, 4)
First 5 samples:
 [['0' '0' 'inversion' '1']
 ['0' '0' 'inversion' '2']
 ['0' '1' 'inversion' '1']
 ['0' '1' 'inversion' '2']
 ['0' '2' 'inversion' '1']]

### Displaying Features for inverted - googlenet ###
Shape of inverted_googlenet_features.npy: (1600, 18432)
First 5 samples:
 [[0.18725644 0.1553466  0.59502125 ... 1.6597188  1.1949198  0.        ]
 [1.903345   1.4736502  0.         ... 0.9204234  0.      